In [68]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [314]:
#  15112020 reconfinement
JE,JI=2.9,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('Rotation:',ROT)

'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=2.0
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
    rhosigv[i,i]=svr2[i]+1.0/svr2[i]
invsigvmat = la.inv(sigvmat)
# print('inverse singular value matrix:',invsigvmat)
# print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

zetau,zetav = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
zetaM,zetaN = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
    else:
        Asqrt[i,i]=0.0
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
# EI format
withrand,wrank = 1,1
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
NT = np.random.randn(nn*2,R)
NT = NT.T 
if R>1:
    Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
else:
    Mrank[:,0],Nrank[:,0]=M1,N1
Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@randomterm*np.sqrt(2*nn)
Nranktildeh = (sigvmat+(grandom**2)*invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*randomterm@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
Nranktilde=Nranktildeh.T 

U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]
Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
if Rc>1:
    Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
    Xtilde_eirec  = Mranktilde@Nranktildeh#/grandom**2
else:
    Xtilde_svdrec = np.reshape(Utilde[:,0],(nn*2,1))@np.reshape(Vtilde[:,0],(1,2*nn))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])#/grandom**2
    Xtilde_eirec  = np.reshape(Mranktilde[:,0],(nn*2,1))@np.reshape(Nranktildeh[0,:],(1,2*nn))#/grandom**2

# test two expressions, using singular values
_,sigvxtilde,_=la.svd(Xtilde)
_,sigveitilde,_=la.svd(Xtilde_eirec)
_,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

print('compare the singular values from')
print('original:',sigvxtilde[:Rc])
print('svd:',sigvsvdtilde[:Rc])
print('ei:',sigveitilde[:Rc])




Rotation: [[-0.95863948  0.28462317]
 [-0.28462317 -0.95863948]]
compare the singular values from
original: [5.72804424]
svd: [8.96806468]
ei: [8.46646293]


In [304]:
print(Asqrt)
print(randomterm)
print(la.matrix_power(invsigvmat,1))

[[0.85444619 0.        ]
 [0.         0.        ]]
[[0.5195399 0.       ]
 [0.        1.       ]]
[[0.20781596 0.        ]
 [0.         1.4070029 ]]


** Large Random Matrix Has Two Outliers **

In [289]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(5,5))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)
RANKP=1
fig=plt.figure(figsize=(5,5))
ax2=fig.add_subplot(111)
ax2.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax2.set_aspect('auto')
ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

RANKP=1
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax2.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax2.set_aspect('auto')
ax2.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax2.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)


In [315]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(5,5))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)



## Figures: Gaussian Distribution

In [80]:
## Gaussian distributions for Utilde,U and Vtilde,V

# numeerically obtain Utildenum, Vtildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Udiffnum,Vdiffnum=Utildenum-U@Asqrt,Vtildenum-V@Asqrt
Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Udiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Udiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Udiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()

# n, bins, patches = ax2.hist(residrvec[:,1], 100, density=True, alpha=0.5)
# stdtest=np.std(residrvec[:,1])
# xxx=np.linspace(-3*stdtest,3*stdtest,1000)
# disttest = np.exp(-(xxx/stdtest)**2/2)/np.sqrt(2*np.pi*stdtest**2)
# # disttest/=np.sum(disttest)
# ax2.plot(xxx,disttest*nn*2/1000)


In [86]:
# numeerically obtain Mtildenum, Ntildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Mtilde_t=Utildenum-np.random.randn(nn*2,R)@la.matrix_power(invsigvmat,3)/np.sqrt(2*nn)
Mtildenum=Mtilde_t*np.sqrt(2*nn)@la.inv(Asqrt)
Mdiffnum =Mtildenum-Mrank@ROT
# Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
# Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Mdiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Mdiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*1)
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Mdiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1])
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()

## Testing Two Representations

In [121]:
#  15112020 reconfinement
JE,JI=1.0,1.45
a,b=0.40,0.50#0.3,0.2#
nvalues,niterations=20,50
aseries = np.linspace(0.0,1.0,nvalues)
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
Singularvalue=2*np.ones((3,2,nvalues,niterations))
for iter in range(niterations):
    for idxa, a in enumerate(aseries):
        J2=np.zeros((2,2))
        J2[:,0],J2[:,1]=JE,-JI
        J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
        J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
        lvec2,svr2,rvech2=la.svd(J2)
        M2 = np.array([[1,1],[1,-1]])
        ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
        # print('Rotation:',ROT)

        '''
        Generate two expressions in large random matrix
        params:
            nn = E/I neuron number
            M/Nrank = low rank structure in EI format
            U/V.    = low rank structyre in SVD format
            M/Nranktilde = low rank EI format for large random matrix
            U/Vtilde = low rank SVD format for large random matrix
            sigvmat: diagonal singular values matrix
            

        '''
        R = 0
        for i in range(2):
            if svr2[i]>1.0:
                R+=1
        nn=600
        if R <2:
            ROT = np.ones((1,1))
        sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
        for i in range(R):
            sigvmat[i,i]=svr2[i]
            rhosigv[i,i]=svr2[i]+1.0/svr2[i]
        invsigvmat = la.inv(sigvmat)
        # print('inverse singular value matrix:',invsigvmat)
        # print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

        zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        # generate rank-2 matrix P without random entries
        P=np.zeros((nn*2,nn*2))
        P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
        P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
        P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
        # use SVD to decompose the low rank matrix
        lvec,svr,rvech=la.svd(P)
        rvec=rvech.T 
        # check the singular value
        # print(svr,svr2)
        if np.sum((svr[:2]-svr2[:])**2)>1e-8:
            
            print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

        alphamat= np.eye(R)-la.matrix_power(invsigvmat,2)
        Asqrt=alphamat.copy()
        for i in range(R):
            Asqrt[i,i]=np.sqrt(Asqrt[i,i])

        # EI format
        withrand,wrank = 1,1
        M1,N1=np.ones(2*nn),np.ones(2*nn)
        N1[:nn],N1[nn:]=JE/nn,-JI/nn
        M2,N2=M1.copy(),N1.copy()
        M2[:nn],M2[nn:]=1,-1
        N2[:nn],N2[nn:]=a/nn,b/nn
        Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
        NT = np.random.randn(nn*2,R)
        NT = NT.T 
        if R>1:
            Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
        else:
            Mrank[:,0],Nrank[:,0]=M1,N1
        Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@invsigvmat*np.sqrt(2*nn)
        Nranktildeh = (sigvmat+invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*invsigvmat@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
        Nranktilde=Nranktildeh.T 



        U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
        U,V=lvec[:,:R],rvec[:,:R]


        Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@invsigvmat,wrank*V@Asqrt+withrand*zetav@invsigvmat

        # add random part in, Xtilde = X+P
        X = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
        Xtilde=X+P

        Xtilde_svdrec = Utilde@(sigvmat+invsigvmat)@Vtilde.T 
        Xtilde_eirec  = Mranktilde@Nranktildeh

        # test two expressions, using singular values
        _,sigvxtilde,_=la.svd(Xtilde)
        _,sigveitilde,_=la.svd(Xtilde_eirec)
        _,sigvsvdtilde,_=la.svd(Xtilde_svdrec)
        if R<2:
            Singularvalue[:,0,idxa,iter]=np.array([sigvxtilde[0],sigveitilde[0],sigvsvdtilde[0]])
        else:
            Singularvalue[0,:,idxa,iter]=sigvxtilde[:R]
            Singularvalue[1,:,idxa,iter]=sigveitilde[:R]
            Singularvalue[2,:,idxa,iter]=sigvsvdtilde[:R]

np.save('Singularvalue17Nov.npy', Singularvalue)  
# print('compare the singular values from')
# print('original:',sigvxtilde[:2])
# print('svd:',sigvsvdtilde[:2])
# print('ei:',sigveitilde[:2])




In [119]:
# np.save('Singularvalue16Nov.npy', Singularvalue)  
fig=plt.figure()
ax1=fig.add_subplot(111)
for iter in range
ax1.plot(aseries,Singularvalue[0,0,:],'r',label=r'numerical $\tilde{\boldsymbol{X}}$')
ax1.plot(aseries,Singularvalue[1,0,:],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
ax1.plot(aseries,Singularvalue[2,0,:],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

ax1.plot(aseries,Singularvalue[0,1,:],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
ax1.plot(aseries,Singularvalue[1,1,:],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
ax1.plot(aseries,Singularvalue[2,1,:],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')


In [49]:
MRT = Mrank@ROT.T/np.sqrt(2*nn)
print('compare MRT and U:',U[:+2,:],MRT[:+2,:]) 
print('compare MRT and U:',U[nn:nn+2,:],MRT[nn:nn+2,:]) 


compare MRT and U: [[-0.02693818 -0.03067574]
 [-0.02693818 -0.03067574]] [[-0.02693818 -0.03067574]
 [-0.02693818 -0.03067574]]
compare MRT and U: [[-0.03067574  0.02693818]
 [-0.03067574  0.02693818]] [[-0.03067574  0.02693818]
 [-0.03067574  0.02693818]]


In [50]:
NRTINV = Nrank@ROT.T@invsigvmat*np.sqrt(2*nn)
print('compare MRT and U:',V[:+2,:],NRTINV[:+2,:]) 
print('compare MRT and U:',V[nn:nn+2,:],NRTINV[nn:nn+2,:]) 

compare MRT and U: [[-0.03536305 -0.02039906]
 [-0.03536305 -0.02039906]] [[-0.03536305 -0.02039906]
 [-0.03536305 -0.02039906]]
compare MRT and U: [[ 0.02039906 -0.03536305]
 [ 0.02039906 -0.03536305]] [[ 0.02039906 -0.03536305]
 [ 0.02039906 -0.03536305]]


In [57]:
print('compare MRT and U:',Utilde[:+2,:],Mranktilde[:+2,:]) 
print('compare MRT and U:',Utilde[nn:nn+2,:],Mranktilde[nn:nn+2,:]) 

compare MRT and U: [[-0.03397748 -0.03438452]
 [-0.02605377 -0.02787703]] [[-0.03397748 -0.03438452]
 [-0.02605377 -0.02787703]]
compare MRT and U: [[-0.03207857  0.03029732]
 [-0.04216421  0.00877207]] [[-0.03207857  0.03029732]
 [-0.04216421  0.00877207]]


In [58]:
print('compare MRT and U:',Vtilde[:+2,:],Nranktilde[:+2,:]) 
print('compare MRT and U:',Vtilde[nn:nn+2,:],Nranktilde[nn:nn+2,:]) 

compare MRT and U: [[-0.02189407 -0.01804848]
 [-0.04999292 -0.01212403]] [[ 0.01241503 -0.00075827]
 [-0.01568382  0.00516618]]
compare MRT and U: [[ 0.00946424 -0.07184279]
 [ 0.01846816 -0.02033009]] [[-0.01032685 -0.04186912]
 [-0.00132293  0.00964357]]
